### Uso de textblob para hacer el análisis de sentimiento de los tweets.

In [1]:
# imports y configuraciones necesarias
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pymongo import MongoClient

import pandas as pd

# !pip install textblob (en caso de no tenerlo instalado previamente)
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

import re
import string
import unicodedata

conf = (SparkSession\
          .builder\
          .appName("twitter")\
          .master("spark://MacBook-Pro-de-Jose.local:7077")\
          .config("spark.io.compression.codec", "snappy")\
          .getOrCreate())  

sc = SparkContext.getOrCreate(conf=conf)

#### Uso del conector Spark y MongoDB

Cogemos los datos cargados en MongoDB, tanto en inglés como en español, almacenados desde el inicio del flujo con Apache Nifi.

Usaremos estos datos para testear el uso de textblob para obtener el sentimiento de los textos.

In [2]:
df_mongo_english = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_english").load()

df_mongo_spanish = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("spark.mongodb.input.uri", "mongodb://localhost:27017/tfm_twitter.tweets_spanish").load()

In [3]:
# nos quedamos en ambos dataframes con la columna texto que es la que nos interesa
df_text_english = df_mongo_english[['text']]
df_text_spanish = df_mongo_spanish[['text']]

In [4]:
df_pandas_spanish = df_text_spanish.toPandas()
df_pandas_english = df_text_english.toPandas()

In [5]:
print("Df spanish: num_rows: %d\tColumnas: %d\n" % (df_pandas_spanish.shape[0], df_pandas_spanish.shape[1]) )
print("Df english: num_rows: %d\tColumnas: %d\n" % (df_pandas_english.shape[0], df_pandas_english.shape[1]) )

Df spanish: num_rows: 79036	Columnas: 1

Df english: num_rows: 322364	Columnas: 1



In [6]:
df_pandas_english.head(10)

,text
0,@EJFC26 Y Messi
1,RT @btsmoonchild64: These group photos deserve...
2,@rehankkhanNDS Overacting *
3,Pay day play day🤑🤑🤑🤑🤑
4,@pandaeyed1 Thank you!
5,RT @liamyoung: Strange that Tony Blair has sud...
6,Hard work puts you where good luck can find you.
7,RT @xCiphxr: When creative kids try playing co...
8,RT @bonang_m: I’m working on one as we speak. ...
9,"RT @akashbanerjee: After #PulwamaAttack, terro..."


Vamos a hacer la prueba con textblob calculando el sentimiento de cada tweet con los tweets en inglés, ya que la librería no funciona igual con otros idiomas, y habría que traducir los textos primero, lo cual es un proceso bastante costoso computacionalmente y reduce después la efectividad de la librería al obtener el sentimiento de un texto.

#### Limpieza de los textos.

Vamos a aplicar a todos los textos del dataframe, técnicas de preprocesado y limpieza de textos para hacer más efectivos y sencillos los análisis de los textos y la clasificación del sentimiento.

In [7]:
# funciones de preprocesado y limpieza de los textos
def limpiar_tweet(tweet):
    # quitamos RT, @nombre_usuario, links y urls, hashtags, menciones, caracteres extraños o emoticonos
    tweet = re.sub('  +', ' ', tweet)
    # eliminar acentos
    tweet = ''.join((c for c in unicodedata.normalize('NFD', tweet) if unicodedata.category(c) != 'Mn'))  
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
    # eliminar "RT", "@usuario", o los enlaces que es información que no sería útil analizar
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub('','',tweet).lower() 
    tweet = re.sub(r'http\S+', '', tweet) 
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet)
    tweet = re.sub(r'[0-9]', '', tweet) 
 
    return tweet 

In [8]:
# aplicamos las funciones a los textos del dataframe, primero hacemos una copia para conservar el original
df_clean_english = df_pandas_english.copy()
df_clean_english['text'] = df_clean_english['text'].apply(limpiar_tweet)

In [9]:
# damos un vistazo a los textos resultantes sobre los que aplicaremos textblob
print("\nDf english:\n", df_clean_english.tail(10))


Df english:
                                                      text
322354  im so excited to see the talents of united com...
322355     dianne abbott in charge of immigration  the...
322356     police are searching for an attacker armed ...
322357                                     your dad      
322358     conservative party leadership faces a membe...
322359              deep purple   highway star  video hq 
322360                                      this edit tho
322361  i am on now one of the  train rides i am takin...
322362    the next   years will be spent slowly watchi...
322363                            ya allahh toothless    


#### Aplicar textblob para obtener el sentimiento de cada texto

TextBlob hace un Análisis de Sentimiento en cualquier texto dado. La propiedad de sentimiento indica la puntuación de sentimiento del texto. Se dan dos puntuaciones: Polaridad y Subjetividad.

La puntuación de polaridad es un valor dentro del rango [-1.0, 1.0] donde un valor negativo indica un texto negativo, y un valor positivo indica que el texto es positivo.

La subjetividad es un valor dentro del rango [0.0, 1.0] donde 0.0 es muy objetivo y 1.0 es muy subjetivo.

In [10]:
# funciones para aplicar textblob al conjunto de textos, y obtener para cada texto el valor de la polaridad.
# Para valores positivos asignamos el valor '2', para valores igual a 0 que serían textos con sentimiento neutro
# asignamos un '1', y por último para valores negativos asignamos el valor '0'.
def analizar_sentimiento(tweet):
    analysis = TextBlob(tweet)
        
    if analysis.sentiment.polarity > 0:
        return 2
    elif analysis.sentiment.polarity == 0:
        return 1
    else:
        return 0

In [11]:
# aplicar textblob a textos en inglés. Lo calculamos sobre los datos originales y los preprocesados.
df_pandas_english['sentiment'] = df_pandas_english['text'].apply(analizar_sentimiento)
df_clean_english['sentiment'] = df_clean_english['text'].apply(analizar_sentimiento)

In [12]:
print("Datos originales en inglés: \n", df_pandas_english.head(10))
print('\n')
print("Datos preprocesados en inglés: \n", df_clean_english.head(10))

Datos originales en inglés: 
                                                 text  sentiment
0                                    @EJFC26 Y Messi          1
1  RT @btsmoonchild64: These group photos deserve...          2
2                        @rehankkhanNDS Overacting *          1
3                              Pay day play day🤑🤑🤑🤑🤑          1
4                             @pandaeyed1 Thank you!          1
5  RT @liamyoung: Strange that Tony Blair has sud...          0
6   Hard work puts you where good luck can find you.          2
7  RT @xCiphxr: When creative kids try playing co...          2
8  RT @bonang_m: I’m working on one as we speak. ...          2
9  RT @akashbanerjee: After #PulwamaAttack, terro...          2


Datos preprocesados en inglés: 
                                                 text  sentiment
0                                            y messi          1
1         these group photos deserve more attention           2
2                                      

In [13]:
# para visualizarlo cambiamos el valor numérico por positivo, neutral y negativo
def get_tweet_sentiment(sentiment): 
    if sentiment > 1: 
        return 'Positivo'
    elif sentiment == 1: 
        return 'Neutral'
    else: 
        return 'Negativo'

df_pandas_english['sentiment'] = df_pandas_english['sentiment'].apply(get_tweet_sentiment)
df_clean_english['sentiment'] = df_clean_english['sentiment'].apply(get_tweet_sentiment)

In [14]:
# ver resultados obtenidos
# vemos los distintos valores que tiene la columna del sentimiento en ambos datasets
print("Recuento valores datos originales en inglés:",pd.value_counts(df_pandas_english['sentiment']))
print('\n')
print("Recuento valores datos procesados en inglés:",pd.value_counts(df_clean_english['sentiment']))

Recuento valores datos originales en inglés: Neutral     144212
Positivo    122267
Negativo     55885
Name: sentiment, dtype: int64


Recuento valores datos procesados en inglés: Neutral     142972
Positivo    124522
Negativo     54870
Name: sentiment, dtype: int64


#### Comentarios sobre los resultados

No se aprecian apenas cambios entre usar textblob con los tweets tal cual se almacenan y recogen, que después de haberlos preprocesado para limpiarlos de caracteres extraños o información que no sea útil para el análisis de sentimiento.

#### Cargamos los csv generados con tweets anotados tanto en inglés como español.

Estos datos ya con sentimiento calculado para cada texto, podremos usarlos para entrenar y testear algún modelo que viene con la librería textblob para clasificar textos, y también si les quitamos los datos del sentimiento para testear el % de acierto de textblob, pasándolos por la librería y luego comparando con el original anotado.

In [10]:
# Los datos de sentimiento vienen con los valores: positivo=2, neutral=1, negativo=0
# cargamos datos en inglés
df_anotados_english = pd.read_csv('./data/df_result_english.csv', sep=',')

print("num_rows: %d\tColumnas: %d\n" % (df_anotados_english.shape[0], df_anotados_english.shape[1]) )
print("Columnas:\n", list(df_anotados_english.columns))

num_rows: 1759315	Columnas: 2

Columnas:
 ['text', 'sentiment']


In [11]:
df_anotados_english.head()

,text,sentiment
0,@VirginAmerica What @dhepburn said.,1.0
1,@VirginAmerica plus you've added commercials t...,2.0
2,@VirginAmerica I didn't today... Must mean I n...,1.0
3,@VirginAmerica it's really aggressive to blast...,0.0
4,@VirginAmerica and it's a really big bad thing...,0.0


Ha cargado los datos de sentimiento como float, posiblemente porque algún valor viene como NaN o similar, sustituimos esos valores por -1 y luego eliminamos esos registros antes de convertir la columna en int.

In [12]:
df_anotados_english = df_anotados_english.fillna(-1)
df_anotados_english = df_anotados_english[df_anotados_english['sentiment']!=-1]

In [13]:
c_float = df_anotados_english.columns[df_anotados_english.dtypes == float]
df_anotados_english[c_float] = df_anotados_english[c_float].astype(int)

In [14]:
df_anotados_english.head()

,text,sentiment
0,@VirginAmerica What @dhepburn said.,1
1,@VirginAmerica plus you've added commercials t...,2
2,@VirginAmerica I didn't today... Must mean I n...,1
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0


In [15]:
# cargamos datos en español
df_anotados_spanish = pd.read_csv('./data/df_result_spanish.csv', sep=',')

print("num_rows: %d\tColumnas: %d\n" % (df_anotados_spanish.shape[0], df_anotados_spanish.shape[1]) )
print("Columnas:\n", list(df_anotados_spanish.columns))

num_rows: 48658	Columnas: 2

Columnas:
 ['text', 'sentiment']


In [16]:
df_anotados_spanish.head(10)

,text,sentiment
0,@PauladeLasHeras No te libraras de ayudar me/n...,1
1,@marodriguezb Gracias MAR,2
2,"Off pensando en el regalito Sinde, la que se v...",0
3,Conozco a alguien q es adicto al drama! Ja ja ...,2
4,Toca @crackoviadeTV3 . Grabación dl especial N...,2
5,Buen día todos! Lo primero mandar un abrazo gr...,2
6,Desde el escaño. Todo listo para empezar #endi...,2
7,Bdías. EM no se ira de puente. Si vosotros os ...,2
8,Un sistema económico q recorta dinero para pre...,2
9,#programascambiados caca d ajuste,0


In [17]:
# generamos otro dataframe con los textos en ingles, quitando del original el sentimiento, para aplicarle textblob
df_noAnotados_english = df_anotados_english[['text']]

In [18]:
# aplicamos al dataframe de textos no anotados la función de limpieza de textos
df_clean_noAnotados = df_noAnotados_english.copy()
df_clean_noAnotados['text'] = df_clean_noAnotados['text'].apply(limpiar_tweet)

In [19]:
df_clean_noAnotados.head(10)

,text
0,what said
1,plus you ve added commercials to the experie...
2,i didn t today must mean i need to take an...
3,it s really aggressive to blast obnoxious e...
4,and it s a really big bad thing about it
5,seriously would pay a flight for seats tha...
6,yes nearly every time i fly vx this ear wo...
7,really missed a prime opportunity for men wi...
8,well i didn t but now i do d
9,it was amazing and arrived an hour early y...


In [46]:
# aplicar textblob para ver el sentimiento que arroja para cada texto
df_clean_noAnotados['sentiment'] = df_clean_noAnotados['text'].apply(analizar_sentimiento)
df_clean_noAnotados.head(10)

,text,sentiment
0,what said,1
1,plus you ve added commercials to the experie...,1
2,i didn t today must mean i need to take an...,0
3,it s really aggressive to blast obnoxious e...,2
4,and it s a really big bad thing about it,0
5,seriously would pay a flight for seats tha...,0
6,yes nearly every time i fly vx this ear wo...,2
7,really missed a prime opportunity for men wi...,2
8,well i didn t but now i do d,1
9,it was amazing and arrived an hour early y...,2


In [47]:
# vemos el recuento de valores 0, 1 y 2 tanto para los datos anotados originales, como los anotados por textblob
print("Recuento valores datos originales en inglés anotados:",pd.value_counts(df_anotados_english['sentiment']))
print("Recuento valores datos pasados por textblob:",pd.value_counts(df_clean_noAnotados['sentiment']))

Recuento valores datos originales en inglés anotados: 2    872864
0    871980
1     14470
Name: sentiment, dtype: int64
Recuento valores datos pasados por textblob: 2    777433
1    614305
0    367576
Name: sentiment, dtype: int64


In [48]:
print("Datos originales en inglés anotados: \n", df_anotados_english.head(10))
print('\n')
print("Datos preprocesados en inglés y anotados mediante textblob: \n", df_clean_noAnotados.head(10))

Datos originales en inglés anotados: 
                                                 text  sentiment
0                @VirginAmerica What @dhepburn said.          1
1  @VirginAmerica plus you've added commercials t...          2
2  @VirginAmerica I didn't today... Must mean I n...          1
3  @VirginAmerica it's really aggressive to blast...          0
4  @VirginAmerica and it's a really big bad thing...          0
5  @VirginAmerica seriously would pay $30 a fligh...          0
6  @VirginAmerica yes, nearly every time I fly VX...          2
7  @VirginAmerica Really missed a prime opportuni...          1
8    @virginamerica Well, I didn't…but NOW I DO! :-D          2
9  @VirginAmerica it was amazing, and arrived an ...          2


Datos preprocesados en inglés y anotados mediante textblob: 
                                                 text  sentiment
0                                       what   said           1
1    plus you ve added commercials to the experie...          1
2

In [49]:
# introducimos en una copia del dataframe original anotado la columna del sentimiento calculado por textblob 
# para después comparar los valores y ver el número de registros donde textblob no ha calculado el mismo 
# valor de sentimiento.
df_anotados_copia = df_anotados_english.copy()
df_anotados_copia['sentiment_textblob'] = df_clean_noAnotados['sentiment']

In [50]:
df_anotados_copia.head(10)

,text,sentiment,sentiment_textblob
0,@VirginAmerica What @dhepburn said.,1,1
1,@VirginAmerica plus you've added commercials t...,2,1
2,@VirginAmerica I didn't today... Must mean I n...,1,0
3,@VirginAmerica it's really aggressive to blast...,0,2
4,@VirginAmerica and it's a really big bad thing...,0,0
5,@VirginAmerica seriously would pay $30 a fligh...,0,0
6,"@VirginAmerica yes, nearly every time I fly VX...",2,2
7,@VirginAmerica Really missed a prime opportuni...,1,2
8,"@virginamerica Well, I didn't…but NOW I DO! :-D",2,1
9,"@VirginAmerica it was amazing, and arrived an ...",2,2


In [51]:
diferencias = df_anotados_copia[df_anotados_copia['sentiment'] != df_anotados_copia['sentiment_textblob']]

In [52]:
print("Número filas datos originales: %d\n" % (df_anotados_english.shape[0]) )
print("Número de valores diferentes: %d\n" % (diferencias.shape[0]) )
print("Porcentaje de registros mal clasificados: %f" % ((diferencias.shape[0]/df_anotados_english.shape[0])*100))

Número filas datos originales: 1759314

Número de valores diferentes: 975540

Porcentaje de registros mal clasificados: 55.450022


Se puede apreciar como hay bastante diferencia en el sentimiento de los datos que vienen ya con el valor del sentimiento anotado (partimos de la suposición de que estos datos están correctamente etiquetados), y el sentimiento de los datos que anotamos su sentimiento mediante el uso de textblob.

Con los datos que tenemos anotados, textblob habría etiquetado erróneamente un 55,45%.

#### Uso de clasificador de textblob

Por último vamos a usar un algoritmo de clasificación que trae textblob (Naive Bayes), entrenándolo con los datos anotados tanto en inglés, como en español.

In [53]:
# tenemos los dataframes cargados en inglés y español ya anotados con su sentimiento
print(df_anotados_english.head(10))
print("\n")
print(df_anotados_spanish.head(10))

                                                text  sentiment
0                @VirginAmerica What @dhepburn said.          1
1  @VirginAmerica plus you've added commercials t...          2
2  @VirginAmerica I didn't today... Must mean I n...          1
3  @VirginAmerica it's really aggressive to blast...          0
4  @VirginAmerica and it's a really big bad thing...          0
5  @VirginAmerica seriously would pay $30 a fligh...          0
6  @VirginAmerica yes, nearly every time I fly VX...          2
7  @VirginAmerica Really missed a prime opportuni...          1
8    @virginamerica Well, I didn't…but NOW I DO! :-D          2
9  @VirginAmerica it was amazing, and arrived an ...          2


                                                text  sentiment
0  @PauladeLasHeras No te libraras de ayudar me/n...          1
1                          @marodriguezb Gracias MAR          2
2  Off pensando en el regalito Sinde, la que se v...          0
3  Conozco a alguien q es adicto al dr

In [54]:
# cogemos un sample de ambos dataframes, para evitar el alto coste computacional de usar tantos datos
df_sample_english = df_anotados_english.sample(frac=0.01, replace=False, random_state=1)
df_sample_spanish = df_anotados_spanish.sample(frac=0.3, replace=False, random_state=1)

In [55]:
# generamos los conjuntos de train y test para entrenar y evaluar el modelo
# convertimos los datos en lista, y repartimos 70% train y 30% test
data_english = df_sample_english[['text', 'sentiment']].values.tolist()
train_index = int(0.70 * len(data_english))
train_english, test_english = data_english[:train_index], data_english[train_index: ]

data_spanish = df_sample_spanish[['text', 'sentiment']].values.tolist()
train_index = int(0.70 * len(data_spanish))
train_spanish, test_spanish = data_spanish[:train_index], data_spanish[train_index: ]

In [56]:
print("Train inglés: %d \t Test inglés: %d" % (len(train_english), len(test_english)))
print("Train español: %d \t Test español: %d" % (len(train_spanish), len(test_spanish)))

Train inglés: 12315 	 Test inglés: 5278
Train español: 10217 	 Test español: 4380


In [57]:
# entrenamos el clasificador para inglés
classifier_english = NaiveBayesClassifier(train_english)

In [58]:
# calcular accuracy del clasificador con tweets en inglés
accuracy_english = classifier_english.accuracy(test_english)
print (accuracy_english) 

0.7254641909814323


In [59]:
# entrenamos el clasificador para español
classifier_spanish = NaiveBayesClassifier(train_spanish)

In [60]:
# calcular accuracy del clasificador con tweets en español
accuracy_spanish = classifier_spanish.accuracy(test_spanish)
print (accuracy_spanish) 

0.819634703196347


In [61]:
# vemos las features más informativas/importantes en inglés
print (classifier_english.show_informative_features(10))

Most Informative Features
            contains(RT) = True                1 : 0      =    157.2 : 1.0
     contains(Microsoft) = True                1 : 2      =    120.6 : 1.0
        contains(Street) = True                1 : 0      =    112.3 : 1.0
       contains(JetBlue) = True                1 : 2      =    111.7 : 1.0
     contains(microsoft) = True                1 : 2      =    111.7 : 1.0
            contains(Ur) = True                1 : 0      =     67.4 : 1.0
        contains(client) = True                1 : 0      =     67.4 : 1.0
     contains(lightning) = True                1 : 0      =     67.4 : 1.0
         contains(Bones) = True                1 : 0      =     67.4 : 1.0
         contains(Robin) = True                1 : 0      =     67.4 : 1.0
None


In [62]:
# vemos las features más informativas/importantes en español
print (classifier_spanish.show_informative_features(10))

Most Informative Features
        contains(riesgo) = True                1 : 2      =     55.8 : 1.0
          contains(cont) = True                2 : 0      =     44.3 : 1.0
        contains(Buenas) = True                2 : 0      =     36.7 : 1.0
            contains(FF) = True                2 : 0      =     30.8 : 1.0
          contains(peor) = True                0 : 2      =     29.5 : 1.0
      contains(denuncia) = True                0 : 2      =     27.7 : 1.0
 contains(AlejandroSanz) = True                2 : 0      =     27.5 : 1.0
         contains(feliz) = True                2 : 0      =     27.2 : 1.0
   contains(interesante) = True                1 : 0      =     27.1 : 1.0
       contains(déficit) = True                0 : 2      =     26.7 : 1.0
None


In [20]:
# vamos a ver si con los textos preprocesados se mejora la precisión del clasificador
# aplicamos las funciones de limpieza a los textos, primero hacemos una copia para conservar el original
df_clean_english = df_anotados_english.copy()
df_clean_english['text'] = df_clean_english['text'].apply(limpiar_tweet)

df_clean_spanish = df_anotados_spanish.copy()
df_clean_spanish['text'] = df_clean_spanish['text'].apply(limpiar_tweet)

In [21]:
# división de los datos preprocesados en train y test 
df_sample_english_2 = df_clean_english.sample(frac=0.01, replace=False, random_state=1)
df_sample_spanish_2 = df_clean_spanish.sample(frac=0.3, replace=False, random_state=1)

data_english_2 = df_sample_english_2[['text', 'sentiment']].values.tolist()
train_index = int(0.70 * len(data_english_2))
train_english_2, test_english_2 = data_english_2[:train_index], data_english_2[train_index: ]

data_spanish_2 = df_sample_spanish_2[['text', 'sentiment']].values.tolist()
train_index = int(0.70 * len(data_spanish_2))
train_spanish_2, test_spanish_2 = data_spanish_2[:train_index], data_spanish_2[train_index: ]

In [24]:
# clasificador de textos en inglés
classifier_english_2 = NaiveBayesClassifier(train_english_2)

accuracy_english_2 = classifier_english_2.accuracy(test_english_2)
print (accuracy_english_2) 

0.7283061765820387


In [26]:
# clasificador de textos en español
classifier_spanish_2 = NaiveBayesClassifier(train_spanish_2)

accuracy_spanish_2 = classifier_spanish_2.accuracy(test_spanish_2)
print (accuracy_spanish_2) 

0.813013698630137


#### Conclusiones del uso de textblob

Lo comentado anteriormente, aplicando Textblob sobre tweets sin etiquetar se acierta un % razonable, pero vemos un % de error al etiquetar el sentimiento bastante elevado. Además el uso con los tweets en español está desaconsejado por temas de coste computacional al tener que traducir los textos, y una posible pérdida de eficiencia si la traducción no es exacta.

Uno de los problemas de textblob es que no consigue del todo acertar con el contexto y subjetividad de los textos, y se ve bastante afectado en sus resultados por ello. Aunque esto es obviamente el gran problema de cualquier técnica de clasificación de textos.

Al usar un clasificador que viene con textblob como Naive Bayes, y entrenarlo con tweets ya anotados, tanto en inglés como en español se ven unos mejores resultados, aunque suponemos que al no poder entrenar con mayor cantidad de datos debido a limitaciones de cómputo e infraestructura no se obtienen los mejores resultados posibles. También recalcar que se vuelve a probar con el clasificador tanto con los datos originales como con los datos preprocesados tras limpiarlos, y se obtienen unos resultados muy similares.

De todos modos la médida para evaluar los modelos usada, la precisión, no es la más adecuada, y cuando probemos con otros modelos y algoritmos de clasificación usaremos otras métricas.